In [ ]:


# Import python packages
import streamlit as st
import pandas as pd  #%%
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session

session = get_active_session()

sales = session.table("SALES").to_pandas()
purchase = session.table("PURCHASES").to_pandas()
invoice = session.table("INVOICE_PURCHASE").to_pandas()
end_inv = session.table("END_INV").to_pandas()
beg_inv = session.table("BEG_INV").to_pandas()
display(beg_inv.head())
display(end_inv.head())



     

In [ ]:

nuniques = {"beg_inv": beg_inv.nunique(), "end_inv": end_inv.nunique()}
display(
    pd.DataFrame(nuniques).T[
        [
            "INVENTORYID",
            "STORE",
            "CITY",
            "BRAND",
            "DESCRIPTION",
            "SIZE",
            "ONHAND",
            "BEGINDATE",
            "ENDDATE",
        ]
    ]
)
print(
    f"BEG_INV BRAND nunique: {beg_inv.BRAND.nunique()}, desc + size nunique: {(beg_inv['DESCRIPTION'] + ' ' + beg_inv['SIZE']).nunique()}??? Might need cleaning"
)
print(
    f"end_inv Brand nunique: {end_inv.BRAND.nunique()}, desc + size nunique: {(end_inv['DESCRIPTION'] + ' ' + end_inv['SIZE']).nunique()}"
)
beg_inv_brand = beg_inv.loc[:]
beg_inv_brand["DESC_SIZE"] = beg_inv_brand["DESCRIPTION"] + " " + beg_inv_brand["SIZE"]
group_desc = (
    beg_inv_brand[["BRAND", "DESC_SIZE"]].groupby("DESC_SIZE")["BRAND"].unique()
)
group_desc.loc[group_desc.apply(len) > 1]

display(sales.head())
display(
    beg_inv.loc[
        (beg_inv["BRAND"] == 1004) & (beg_inv["INVENTORYID"] == "1_HARDERSFIELD_1004")
        ]
)
# ? Inventory ID = store_city_brand, Brand = description + Size, With Inventory ID we can find how many onhand the inventory have at the beginning and end.
print(sales.CLASSIFICATION.unique())